In [2]:
# import packages
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import time

2024-03-26 16:40:17.689537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow_datasets'

In [3]:
# Datasets
train_paths = [
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_AREA/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BICUBIC/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BILINEAR/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_GAUSSIAN/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS5/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_MITCHELLCUBIC/train',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_ NEAREST_NEIGHBOR/train'   
]
test_paths = [
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_AREA/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BICUBIC/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_BILINEAR/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_GAUSSIAN/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS3/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_LANCZOS5/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_MITCHELLCUBIC/test',
    '/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16_ NEAREST_NEIGHBOR/test'   
]
csv_names = [
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/AREA.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/BICUBIC.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/BILINEAR.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/GAUSSIAN.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/LANCZOS3.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/LANCZOS5.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/MITCHELLCUBIC.csv',
    '/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/NEAREST_NEIGHBOR.csv'
]

#train = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/train')
#test  = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/test')

def preprocess(example):
    image = example['image']
    image.set_shape([16, 16, 1])
    label = example['label']
    label = tf.one_hot(label, depth=100)  # One-hot encode the labels
    return image, label

def format_set(train_set, test_set):
    # format and cache
    train_set_formatted = train_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train_set_formatted = train_set_formatted.cache()
    train_set_formatted = train_set_formatted.batch(256)
    train_set_formatted = train_set_formatted.prefetch(tf.data.AUTOTUNE)
    test_set_formatted = test_set.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    test_set_formatted = test_set_formatted.batch(256)
    test_set_formatted = test_set_formatted.cache()
    test_set_formatted = test_set_formatted.prefetch(tf.data.AUTOTUNE)
    return train_set_formatted, test_set_formatted

/home/ubuntu/approximate_computing_in_CNN/app-small_network/results/AREA.csv


In [9]:
'''
# Get the first batch of images and labels from the training dataset
images, labels = next(iter(train))

# Create a figure
plt.figure(figsize=(10,10))

# Plot 25 images
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # Reshape the image
    image = np.reshape(images[i], (16, 16))
    plt.imshow(image, cmap=plt.cm.binary)
    # Get the label for the image
    label = np.argmax(labels[i])
    plt.xlabel(label)
plt.show()
'''

'\n# Get the first batch of images and labels from the training dataset\nimages, labels = next(iter(train))\n\n# Create a figure\nplt.figure(figsize=(10,10))\n\n# Plot 25 images\nfor i in range(25):\n    plt.subplot(5,5,i+1)\n    plt.xticks([])\n    plt.yticks([])\n    plt.grid(False)\n    # Reshape the image\n    image = np.reshape(images[i], (16, 16))\n    plt.imshow(image, cmap=plt.cm.binary)\n    # Get the label for the image\n    label = np.argmax(labels[i])\n    plt.xlabel(label)\nplt.show()\n'

In [3]:
# Keeping time
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

In [4]:
# Model 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Softmax()
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['accuracy']
)

In [5]:
#train = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/train')
#test  = tf.data.Dataset.load('/home/ubuntu/tensorflow_datasets/cifar100_grey_16x16/test')

for i in range(csv_names.len()):
    # fetch datasets
    train = tf.data.Dataset.load(train_paths[i])
    test  = tf.data.Dataset.load(test_paths[i])
    train, test = format_set(train, test)

    # Train
    history = model.fit(train, epochs=100, validation_data=test, callbacks=[time_callback])

    # Convert the history.history dict to a pandas DataFrame
    hist_df = pd.DataFrame(history.history)

    # Add epoch times
    hist_df['time'] = time_callback.times

    # Save to csv
    hist_df.to_csv(csv_names[i])

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0184 - loss: 0.0099 - val_accuracy: 0.0524 - val_loss: 0.0098
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0599 - loss: 0.0098 - val_accuracy: 0.0810 - val_loss: 0.0097
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0859 - loss: 0.0097 - val_accuracy: 0.0985 - val_loss: 0.0096
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1020 - loss: 0.0096 - val_accuracy: 0.1090 - val_loss: 0.0096
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1150 - loss: 0.0095 - val_accuracy: 0.1174 - val_loss: 0.0095
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1266 - loss: 0.0095 - val_accuracy: 0.1241 - val_loss: 0.0095
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.1358 - loss: 0.0094 - val_accuracy: 0.1290 - val_loss: 0.0095
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.1436 - loss: 0.0093 - val_accuracy: 0.